In [ ]:
"""
Problemillas que veo:
- como inciar el pipeline
- Añadir un input de "mes" para modeloScoring.py
- Hay que hacer los dockerfile y añadir la imagen al yaml
"""

"""
1) Estudiar filtrado de mes (modelo scoring, columna, ...)
3) Añadir mes al pipeline??
4) Modificar yaml de los archivos cambiados (modeloscoring o preprocesamiento2)
5) Estructurar los archivos
6) Generar los dockerfile (modificar los de moragon)
7) Añadir imagenes de los docker a los yaml y modificar los yaml (path)

In [22]:
import kfp
from typing import NamedTuple
from kfp import dsl
"""
import tensorflow as tf
from tensorflow import keras

from tensorflow.python.framework import dtypes
from tensorflow import feature_column
"""

from kfp.dsl import  pipeline, component, Artifact, Dataset, Input, Metrics, Model, Output, InputPath, OutputPath

# Descargar archivos originales

In [13]:
import pandas as pd
df_original_bananas = pd.read_csv("gs://kubeflow-bucket-prueba/banana_quality.csv")
print(df_original_bananas)

          Size    Weight  Sweetness  Softness  HarvestTime  Ripeness  \
0    -1.924968  0.468078   3.077832 -1.472177     0.294799  2.435570   
1    -2.409751  0.486870   0.346921 -2.495099    -0.892213  2.067549   
2    -0.357607  1.483176   1.568452 -2.645145    -0.647267  3.090643   
3    -0.868524  1.566201   1.889605 -1.273761    -1.006278  1.873001   
4     0.651825  1.319199  -0.022459 -1.209709    -1.430692  1.078345   
...        ...       ...        ...       ...          ...       ...   
7995 -6.414403  0.723565   1.134953  2.952763     0.297928 -0.156946   
7996  0.851143 -2.217875  -2.812175  0.489249    -1.323410 -2.316883   
7997  1.422722 -1.907665  -2.532364  0.964976    -0.562375 -1.834765   
7998 -2.131904 -2.742600  -1.008029  2.126946    -0.802632 -3.580266   
7999 -2.660879 -2.044666   0.159026  1.499706    -1.581856 -1.605859   

       Acidity Quality  
0     0.271290    Good  
1     0.307325    Good  
2     1.427322    Good  
3     0.477862    Good  
4     2.81

# Creación de datos

In [14]:
# Generar datos
# Se lo dejamos a orianna :)

import pandas as pd
from google.cloud import storage
from io import StringIO
from datetime import datetime
import generarDatos

df_extendido = generarDatos.generar_datos_df(df_original_bananas, 1000000)

print("\n==> datos creados con éxito") 

def guardar_df_en_gcs(df, bucket_name, blob_name=None):
    """Guardar un DataFrame de pandas en un archivo CSV en un blob en Google Cloud Storage."""
    client = storage.Client()
    bucket = client.bucket(bucket_name)

    # Generar un nombre de blob si no se proporciona uno
    if blob_name is None:
        now = datetime.now()
        mes = now.strftime("%m")
        fecha = now.strftime("%Y%m%d")
        hora = now.strftime("%H%M")
        
        blob_name = f"datos_{mes}_{fecha}_{hora}.csv"

    # Convertir DataFrame a formato CSV
    csv_data = df.to_csv(index=False)

    # Cargar el archivo CSV en el blob
    blob = bucket.blob(blob_name)
    blob.upload_from_string(csv_data, content_type='text/csv')

    print(f"\n==> DataFrame guardado en el blob '{blob_name}' en el bucket '{bucket_name}'.")
    
guardar_df_en_gcs(df_extendido, 'kubeflow-bucket-prueba') # Nombre de mi bucket

    



==> datos creados con éxito

==> DataFrame guardado en el blob 'datos_05_20240509_0941.csv' en el bucket 'kubeflow-bucket-prueba'.


# Definición del pipeline


In [37]:
from typing import NamedTuple
import kfp
from kfp import dsl
from kfp.v2.dsl import InputPath, OutputPath
from preprocesamiento2 import preprocesamiento2_component
from preprocesamiento1 import preprocesamiento_component
from modeloscoring import modeloscoring_component
from kfp.gcp import components as gcp_components


@dsl.pipeline(
    name='Preprocesamiento Pipeline',
    description='Pipeline para preprocesar datos'
)
def pipeline_completo(
    input_csv: InputPath('CSV'),
    output_csv: OutputPath('CSV'),
    mes: int
):
    # Leer el CSV de entrada
    read_csv_op = gcp_components.load_file(
        input_path=input_csv,
        name='Leer CSV de entrada'
    )

    # Ejecutar el componente de preprocesamiento 1
    preprocesamiento_op = preprocesamiento_component(read_csv_op.output)
    
    # Ejecutar el segundo componente de preprocesamiento
    preprocesamiento2_op = preprocesamiento2_component(preprocesamiento_op.output)

    # Ejecutar el componente de scoring de modelos
    scoring_op = modeloscoring_component(preprocesamiento2_op.output, mes) 

    # Convertir el DataFrame de predicciones a CSV y guardarlo en GCS
    save_csv_op = gcp_components.write_to_gcs(
        input_data=scoring_op.output,
        output_path=output_csv,
        name='Guardar CSV de salida'
    )

    # Definir dependencias entre operaciones
    save_csv_op.after(preprocesamiento_op)

TypeError: Missing type annotation for argument: df

In [ ]:
mes_input = input("Indique el mes (número): ")

input_bucket = 'gs://kubeflow-bucket-prueba'
input_csv_file = input("Inserte el nombre del archivo CSV de los datos extendidos de banana: ")

# Crear una instancia del cliente de KFP para interactuar con el Kubeflow Pipelines en GCP
client = kfp.Client()

# Compilar y ejecutar el pipeline en Kubeflow Pipelines en GCP
client.create_run_from_pipeline_func(
    pipeline_completo,
    arguments={
        'input_csv': input_bucket + '/' + input_csv_file,
        'output_csv': 'gs://kubeflow-bucket-prueba/scoring_{mes}.csv',
        'mes'= mes_input
    }
)